In [61]:
import numpy as np
import math
import pandas as pd

In [62]:
def modify_covariance_1(cov, lamba, S,num, num_all):
    return \
        ((1 - lamba) * num * cov + lamba * num_all * S) \
        / ((1 - lamba) * num + lamba * num_all)
def modify_corvariance_2(cov, ep) :
    trace = cov.trace() / cov.shape[0]
    Id = np.eye(cov.shape[0])
    return (1 - ep) * cov + ep * trace * Id

def S(cov_1,cov_2,cov_3,n1,n2,n3) :
    return cov_1 * n1 / (n1 + n2+ n3) + cov_2 * n2 \
           / (n1 + n2 + n3) + cov_3 * n3 / (n1 + n2 + n3)
    
def rda_modify(cov ,lamba, S, num , num_all, ep) :
    cov_m = modify_covariance_1(cov, lamba, S , num, num_all)
    return modify_corvariance_2(cov_m , ep)


def get_qda_cor_matrix (train, mu) :
    # print(len(train))
    size = train[0].shape[0]
    zero: np.ndarray = np.random.normal(0, 0.01, size=(size,size))
    i = 1
    for vec in train : 
        if i == 1 :
            # print(type(vec - mu))
            # print(type(zero))
            i += 1
        zero += np.multiply((vec - mu), vec - mu)
    return zero / len(train)

def get_qda_norm_attribute (train) : 
    # print(train)
    init_vec = np.zeros(train[0].shape[0])
    for vec in train :
        init_vec += vec
    return init_vec / len(train)

def get_sample_covariance_matrix(sample_data):
    mean_i = get_qda_norm_attribute(sample_data)
    sample_data_1 = sample_data
        #print type(sample_data_1)
    covMatrix = get_qda_cor_matrix(sample_data_1, mean_i)
        #covMatrix = np.cov(sample_data)
    return covMatrix,mean_i


In [63]:
def encoding_(dict : dict, idx_to_stuff,stuff, ty=float) :
    num = len(dict.items())
    if dict.__contains__(stuff) : # 包含这个数值 则不进行处理 返回code
        return ty(dict[stuff])
    else : 
        dict[stuff] = num
        idx_to_stuff[num] = stuff
        return ty(num)

def encode_raw_data(raw_data_set : pd.DataFrame, di= None) :
    out_matrix = np.zeros(raw_data_set.shape)
    # size = raw_data_set.items()
    # print(size)
    ret = []
    for ind in range(raw_data_set.shape[1]) :
        col = raw_data_set[ind]
        dic = dict()
        idx_dic = dict()
        if di:
            dic,idx_dic = di[ind] 
        idx = 0
        for elem in col : 
            elem_code  = encoding_(dic, idx_dic, elem)
            out_matrix[idx,ind] = elem_code
            idx += 1
        ret.append((dic, idx_dic))
    return out_matrix, ret

def discriminant_function(x, covMatrix, mean_i,p_i):
        cov_inverse = np.linalg.pinv(covMatrix)
        a = x - mean_i
        b = np.dot(a.T, cov_inverse)
        c = -np.dot(b, a)
        discriminant_function_value =  math.log(p_i) - \
                                       c / 2 - 1/2 * \
                                       math.log(np.abs
                                                (np.linalg.det(covMatrix)))
        return discriminant_function_value

In [64]:
import sys
sys.path.append("..")
raw_data_test = pd.read_csv("C:\\Users\\coco1\\PycharmProjects\\PRHM01\\04\\ds04\\iris.data", dtype=str,sep=",", header=None)
raw_data = pd.read_csv("C:\\Users\\coco1\\PycharmProjects\\PRHM01\\04\\ds04\\bezdekIris.data", dtype=str,sep=",", header=None)
# raw_data = pd.read_csv("C:\\Users\\coco1\\PycharmProjects\\PRHM01\\03\\ds03\\adult.data", header=None, dtype=str)
# train,di_raw = encode_raw_data(raw_data)
# test,di_raw = encode_raw_data(raw_data_test, di_raw)
train = raw_data.sample(frac=1).values
test = raw_data_test.sample(frac=1).shift().values

In [65]:
train_set = np.array(train[:,0:-1], dtype=float)
train_label = np.array(train[:,-1], dtype=str)
test_set = np.array(test[:,0:-1], dtype=float)
test_label = np.array(test[:,-1], dtype=str)

In [73]:
def classify (td, tl) :
    type1_train_data = []
    type2_train_data = []
    type3_train_data = []
    num_1 = 0
    num_2 = 0
    num_3 = 0
    for i in range(len(tl)) :
        if str(tl[i]) == "Iris-setosa" : 
            type1_train_data.append(td[i])
            num_1 += 1
        if str(tl[i]) == "Iris-versicolor" : 
            type2_train_data.append(td[i])
            num_2 += 1
        if str(tl[i]) == "Iris-virginica" : 
            type3_train_data.append(td[i])
            num_3 += 1
    return type1_train_data, type2_train_data, type3_train_data, \
           num_1, num_2 , num_3
    # print(str(train_label[i]))
def split_into_folds(ds, label,folds_n):
    li = []
    length = len(ds)
    num = int(length / folds_n)
    start = 0
    end = num
    for i in range(folds_n) :
        if i == folds_n - 1 : end = length - 1
        test = []
        va = []
        test_l = []
        va_l = []
        for j in range(length) :
            if end >= j >= start :
                test.append(ds[j])
                test_l.append(label[j])
            else: 
                va.append(ds[j])
                va_l.append(label[j])
        li.append([test, va, test_l, va_l])
        start += num
        end += num
    return li
def evaluate (test, tl, lamda, ep, train, trl) :
    t1,t2,t3,n1,n2,n3 = classify(train, trl)
    if n1 == 0 or n2 == 0 or n3 == 0 : return 0
    # print(n1,n2,n3)
    cov_1,m_1 = get_sample_covariance_matrix(t1)
    # m_1 = get_qda_norm_attribute(t1)
    cov_2,m_2 = get_sample_covariance_matrix(t2)
    cov_3,m_3 = get_sample_covariance_matrix(t3)
    # m_2 = get_qda_norm_attribute(t2)
    s = S(cov_1,cov_2,cov_3, n1,n2,n3)
    cov_m_1 = rda_modify(cov_1, lamda, s, n1 ,n1+n2+n3, ep)
    cov_m_2 = rda_modify(cov_2, lamda, s, n2 ,n1+n2+n3, ep)
    cov_m_3 = rda_modify(cov_3, lamda, s, n3 ,n1+n2+n3, ep)
    score = 0
    ind = 0
    for x in test : 
        j_1 = discriminant_function(x, cov_m_1, m_1, n1 / (n1+n2+n3))
        j_2 = discriminant_function(x, cov_m_2, m_2, n2 / (n1+n2+n3))
        j_3 = discriminant_function(x, cov_m_3, m_3, n3 / (n1+n2+n3))
        if j_1 > j_2 :
            if j_1 > j_3 :
                re = "Iris-setosa"
            else : re = "Iris-virginica"
        else : 
            if j_2 > j_3 :re = "Iris-versicolor"
            else :  re = "Iris-virginica"
        if re == str(tl[ind]) : 
            score += 1
        print(re, tl[ind])
        ind += 1
       
    return score

In [74]:
#cross validation :
import matplotlib as plt
def get_parameter_by_cross_validate(ds, label, folds_n):
        lamba_list= []
        ep_list = []
        result_score = []
        li = split_into_folds(ds, label, folds_n)
        for e in range(10):
            for lm in range(10):
                scores = []
                lamba = 0.1 * float(lm)
                ep = 0.1 * float(e)
                for i in range(folds_n):
                    training_data_i = li[i][0]
                    test_data_i = li[i][1]
                    train_data_label = li[i][2]
                    test_data_label = li[i][3]
                    score = evaluate\
                        (test_data_i, 
                         test_data_label,
                         lamba , 
                         ep, 
                         training_data_i,
                         train_data_label)
                    #print score
                    scores.append(score)
                score_mean = np.mean(scores)
                # print score_mean
                result_score.append(score_mean)
                lamba_list.append(lamba)
                ep_list.append(ep)
        l = lamba_list[result_score.index(max(result_score))]
        e = ep_list[result_score.index(max(result_score))]
        return l, e

In [68]:

l,e = get_parameter_by_cross_validate(train_set, train_label, 10)



In [75]:
print(l,e)

score = evaluate(test_set, 
                test_label,
                l , 
                e, 
                train_set,
                train_label)
print(score)

0.8 0.0
Iris-virginica nan
Iris-setosa Iris-setosa
Iris-versicolor Iris-versicolor
Iris-versicolor Iris-versicolor
Iris-virginica Iris-virginica
Iris-virginica Iris-virginica
Iris-virginica Iris-virginica
Iris-virginica Iris-virginica
Iris-virginica Iris-virginica
Iris-versicolor Iris-versicolor
Iris-setosa Iris-setosa
Iris-versicolor Iris-versicolor
Iris-versicolor Iris-versicolor
Iris-versicolor Iris-versicolor
Iris-versicolor Iris-versicolor
Iris-setosa Iris-setosa
Iris-virginica Iris-virginica
Iris-setosa Iris-setosa
Iris-virginica Iris-virginica
Iris-versicolor Iris-versicolor
Iris-virginica Iris-virginica
Iris-setosa Iris-setosa
Iris-versicolor Iris-versicolor
Iris-virginica Iris-virginica
Iris-setosa Iris-setosa
Iris-virginica Iris-virginica
Iris-versicolor Iris-versicolor
Iris-virginica Iris-virginica
Iris-setosa Iris-setosa
Iris-virginica Iris-virginica
Iris-virginica Iris-virginica
Iris-versicolor Iris-versicolor
Iris-virginica Iris-virginica
Iris-versicolor Iris-versicolor
I